# Latent Dirichlet Allocation (LDA)

Sklearn example from https://medium.com/mlreview/topic-modeling-with-scikit-learn-e80d33668730

In [22]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

### Load data from the 20 news groups data

In [23]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print(documents[0])

Well i'm not sure about the story nad it did seem biased. What
I disagree with is your statement that the U.S. Media is out to
ruin Israels reputation. That is rediculous. The U.S. media is
the most pro-israeli media in the world. Having lived in Europe
I realize that incidences such as the one described in the
letter have occured. The U.S. media as a whole seem to try to
ignore them. The U.S. is subsidizing Israels existance and the
Europeans are not (at least not to the same degree). So I think
that might be a reason they report more clearly on the
atrocities.
	What is a shame is that in Austria, daily reports of
the inhuman acts commited by Israeli soldiers and the blessing
received from the Government makes some of the Holocaust guilt
go away. After all, look how the Jews are treating other races
when they got power. It is unfortunate.



### Prepare the data for LDA

In [24]:
num_features = 1000

# LDA can only uses raw term counts
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=num_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)

In [4]:
tf_feature_names = tf_vectorizer.get_feature_names() #theses are the words in our bag of words

### Build the LDA model

In [40]:
num_topics = 10

# Run LDA
lda = LatentDirichletAllocation(n_components=num_topics, max_iter=5, learning_method='online',random_state=0, n_jobs=-1)
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [43]:
temp = lda.components_[0]
print(temp.shape)
temp.argsort()[:10].shape

(1000,)


TypeError: 'str' object cannot be interpreted as an integer

### Display the top ten words for each topic

In [7]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-num_top_words - 1:-1]]))

num_top_words = 10
display_topics(lda, tf_feature_names, num_top_words)

Topic 0:
people government gun armenian said armenians states war israel state
Topic 1:
people don like make right point know think use just
Topic 2:
space program db output data entry nasa use build science
Topic 3:
key encryption chip government use keys public clipper security law
Topic 4:
edu file com available information mail ftp files list send
Topic 5:
god people jesus does believe say think christian bible true
Topic 6:
windows use drive thanks does card problem like know using
Topic 7:
ax max b8f g9v a86 pl 145 1d9 0t 34u
Topic 8:
just don like think know good time didn going ve
Topic 9:
10 00 20 15 25 12 14 11 17 16


### Model evaluation

Model [perplexity](https://en.wikipedia.org/wiki/Perplexity) is often used in LDA to evaluate how well a model predicts a sample.

In [8]:
print("Model perplexity: {0:0.3f}".format(lda.perplexity(tf)))

Model perplexity: 265.162


## Show how to do LDA in gensim

Example from https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

If you don't already have gensim installed:  
`$ pip install -U gensim`

### Imports

In [9]:
import pandas as pd
import numpy as np

#gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import gensim.corpora as corpora
from gensim.models import CoherenceModel

#nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
# nltk.download('wordnet')

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

### Load data

In [10]:
data = pd.read_csv('data/abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

### Preprocess data

In [11]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [12]:
stemmer = SnowballStemmer('english')
processed_docs = documents['headline_text'].map(preprocess)

In [13]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

### Bag of words

In [14]:
#create dictionary
id2word = gensim.corpora.Dictionary(processed_docs)

#create corpus
texts = processed_docs

#Term Document Frequency
bow_corpus = [id2word.doc2bow(text) for text in texts]

In [15]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in bow_corpus]

[[('broadcast', 1), ('communiti', 1), ('decid', 1), ('licenc', 1)],
 [('awar', 1), ('defam', 1), ('wit', 1)],
 [('call', 1), ('infrastructur', 1), ('protect', 1), ('summit', 1)],
 [('aust', 1), ('rise', 1), ('staff', 1), ('strike', 1)],
 [('strike', 1), ('affect', 1), ('australian', 1), ('travel', 1)],
 [('ambiti', 1), ('jump', 1), ('olsson', 1), ('tripl', 1), ('win', 1)],
 [('antic', 1), ('barca', 1), ('break', 1), ('delight', 1), ('record', 1)],
 [('aussi', 1),
  ('match', 1),
  ('memphi', 1),
  ('qualifi', 1),
  ('stosur', 1),
  ('wast', 1)],
 [('aust', 1), ('address', 1), ('council', 1), ('iraq', 1), ('secur', 1)],
 [('australia', 1), ('lock', 1), ('timet', 1)],
 [('iraq', 1), ('australia', 1), ('contribut', 1), ('million', 1)],
 [('barca', 1), ('record', 1), ('birthday', 1), ('celebr', 1), ('robson', 1)],
 [('ahead', 1), ('bathhous', 1), ('plan', 1)],
 [('championship', 1), ('cycl', 1), ('hop', 1), ('launceston', 1)],
 [('plan', 1), ('boost', 1), ('paroo', 1), ('suppli', 1), ('wat

In [16]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=id2word, passes=2, workers=2)

### View topics in the LDA model

In [17]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[bow_corpus]

[(0,
  '0.040*"trump" + 0.023*"warn" + 0.021*"countri" + 0.017*"hour" + '
  '0.014*"lose" + 0.014*"victoria" + 0.012*"beat" + 0.011*"street" + '
  '0.011*"john" + 0.011*"storm"'),
 (1,
  '0.061*"say" + 0.022*"elect" + 0.015*"labor" + 0.014*"report" + 0.014*"deal" '
  '+ 0.012*"news" + 0.012*"polit" + 0.011*"australia" + 0.010*"parti" + '
  '0.010*"build"'),
 (2,
  '0.029*"attack" + 0.026*"kill" + 0.019*"jail" + 0.017*"dead" + 0.016*"miss" '
  '+ 0.015*"woman" + 0.014*"life" + 0.013*"sentenc" + 0.012*"leader" + '
  '0.010*"year"'),
 (3,
  '0.042*"polic" + 0.021*"school" + 0.018*"melbourn" + 0.016*"canberra" + '
  '0.012*"student" + 0.010*"investig" + 0.010*"find" + 0.010*"violenc" + '
  '0.009*"arrest" + 0.009*"bodi"'),
 (4,
  '0.025*"nation" + 0.019*"interview" + 0.018*"rural" + 0.017*"crash" + '
  '0.015*"die" + 0.015*"chang" + 0.014*"concern" + 0.013*"road" + '
  '0.013*"train" + 0.013*"park"'),
 (5,
  '0.022*"hous" + 0.018*"live" + 0.016*"tasmanian" + 0.016*"donald" + '
  '0.015*"tu

### Compute Model Perplexity and Coherence Score (interpretability of the model)



In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.249487917632548

Coherence Score:  0.2232699850158148


### Visualize the topics-keywords

In [20]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
vis

/Users/kayla.thomas/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.156050  0.216408       1        1  13.411084
6     -0.249700 -0.041405       2        1  11.392888
4      0.166396  0.112568       3        1   9.977107
3      0.180193 -0.046459       4        1   9.828695
7     -0.200433  0.107932       5        1   9.616330
8      0.086564 -0.290531       6        1   9.528026
1      0.009896  0.111256       7        1   9.334035
5      0.016694  0.123263       8        1   9.322736
2      0.077571 -0.255980       9        1   9.154254
0     -0.243232 -0.037051      10        1   8.434839, topic_info=      Category          Freq        Term         Total  loglift  logprob
411    Default  28736.000000         say  28736.000000  30.0000  30.0000
241    Default  29264.000000       polic  29264.000000  29.0000  29.0000
7880   Default  17188.000000       trump  17188.000000  28.0000  28.0000
326    Default  17552.000000       charg  17552.000000  27.0000  27.0000
2543   Default  19194.000000      govern  19194.000000  26.0000  26.0000
217    Default  16331.000000       court  16331.000000  25.0000  25.0000
81     Default  13569.000000      attack  13569.000000  24.0000  24.0000
2104   Default  13796.000000  queensland  13796.000000  23.0000  23.0000
307    Default  12658.000000      murder  12658.000000  22.0000  22.0000
193    Default  12453.000000      nation  12453.000000  21.0000  21.0000
541    Default  11800.000000        kill  11800.000000  20.0000  20.0000
1415   Default  11797.000000       south  11797.000000  19.0000  19.0000
1142   Default  16794.000000        year  16794.000000  18.0000  18.0000
38     Default  26782.000000   australia  26782.000000  17.0000  17.0000
247    Default  10897.000000       north  10897.000000  16.0000  16.0000
429    Default  11697.000000       world  11697.000000  15.0000  15.0000
784    Default  13927.000000       elect  13927.000000  14.0000  14.0000
433    Default  10662.000000        face  10662.000000  13.0000  13.0000
288    Default  10524.000000        hous  10524.000000  12.0000  12.0000
146    Default  13725.000000       death  13725.000000  11.0000  11.0000
351    Default   9693.000000        warn   9693.000000  10.0000  10.0000
1070   Default  10237.000000      school  10237.000000   9.0000   9.0000
587    Default  10058.000000        test  10058.000000   8.0000   8.0000
1354   Default   9622.000000   interview   9622.000000   7.0000   7.0000
1136   Default   9424.000000      market   9424.000000   6.0000   6.0000
425    Default   8917.000000     countri   8917.000000   5.0000   5.0000
873    Default  13108.000000    melbourn  13108.000000   4.0000   4.0000
48     Default  14240.000000        plan  14240.000000   3.0000   3.0000
334    Default   8945.000000        jail   8945.000000   2.0000   2.0000
1298   Default   9255.000000       rural   9255.000000   1.0000   1.0000
...        ...           ...         ...           ...      ...      ...
1471   Topic10   7282.201172        hour   7283.038574   2.4727  -4.0672
340    Topic10   5987.870605        lose   5988.708008   2.4727  -4.2629
1539   Topic10   5746.905762    victoria   5747.743164   2.4727  -4.3039
1677   Topic10   4941.709961        beat   4942.547363   2.4726  -4.4549
2977   Topic10   4762.767090      street   4763.604492   2.4726  -4.4918
3130   Topic10   4687.719238        john   4688.556641   2.4726  -4.5076
1884   Topic10   4499.011719       storm   4499.849121   2.4726  -4.5487
3042   Topic10   4404.491699        drum   4405.329102   2.4726  -4.5700
608    Topic10   4256.917969      season   4257.755371   2.4726  -4.6040
384    Topic10   4215.538574        give   4216.375977   2.4726  -4.6138
666    Topic10   3799.178711     weather   3800.016113   2.4726  -4.7178
552    Topic10   3789.695068       shark   3790.532471   2.4726  -4.7203
1092   Topic10   3720.708252        play   3721.546387   2.4726  -4.7387
2564   Topic10   3707.36